In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('DataFrame_3G_After_Cleaning.csv')

In [3]:
import plotly.express as px
from sklearn.ensemble import IsolationForest
import plotly.graph_objects as go

In [7]:
x = df['CS CSSR'].apply(lambda x: [x]).to_list()
iso_forest = IsolationForest(n_estimators=100,
                             max_samples="auto",
                             contamination=0.025,
                             )
iso_forest.fit(x)
y_pred = iso_forest.predict(x)
y_pred = [1 if x == -1 else 0 for x in y_pred]
y_pred[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [8]:
df["anomaly"] = y_pred
df.head()

,index,Date,RBS Id,TRAFFIC PS(DL+UL)(Gbits),HSDPA Data Volume,HSUPA Data Volume,Cell Traffic Volume DL Speech (Erlang),CS CSSR,CS DROP Rate,PS Call Setup SR,PS DROP Rate 2,Availability,Soft Handover,Irat CS Handover SR,IRAT PS 2,throughput,anomaly
0,0,16-12-21,0,74.39,66.74,7.65,156.83,99.69,0.77,95.94,1.72,100.0,99.86,93.94,100.00,755.79,0
1,1,17-12-21,0,65.41,58.40,7.01,107.82,99.26,0.97,95.40,2.37,100.0,99.67,97.98,100.00,755.79,0
2,2,18-12-21,0,65.67,58.93,6.74,94.38,99.26,1.09,94.97,3.03,100.0,99.57,96.43,0.00,755.79,0
3,3,19-12-21,0,57.79,50.91,6.87,101.48,99.08,1.23,95.43,2.60,100.0,99.72,93.14,100.00,755.79,0
4,4,20-12-21,0,79.74,70.04,9.69,163.22,99.65,0.76,95.16,1.61,100.0,99.86,94.23,66.67,755.79,0


In [9]:
iso_anomaly_df = pd.DataFrame(df)
iso_anomaly_df = iso_anomaly_df.loc[iso_anomaly_df['anomaly'] == 1]
iso_anomaly_df.head()

,index,Date,RBS Id,TRAFFIC PS(DL+UL)(Gbits),HSDPA Data Volume,HSUPA Data Volume,Cell Traffic Volume DL Speech (Erlang),CS CSSR,CS DROP Rate,PS Call Setup SR,PS DROP Rate 2,Availability,Soft Handover,Irat CS Handover SR,IRAT PS 2,throughput,anomaly
23,23,01-08-22,0,76.68,67.55,9.14,123.39,98.87,3.00,94.32,3.23,97.75,99.42,96.14,40.0,628.69,1
24,24,01-09-22,0,72.46,61.93,10.53,89.79,98.41,3.48,93.11,4.73,96.95,98.69,92.60,100.0,490.41,1
25,25,01-10-22,0,64.63,57.21,7.42,110.09,98.29,3.87,89.49,3.79,97.88,99.60,95.49,100.0,670.35,1
31,31,16-01-22,0,61.30,54.08,7.22,80.83,98.85,2.67,94.80,3.69,98.14,99.39,96.33,100.0,660.05,1


In [10]:
fig = px.line(df,x='index', y='CS CSSR', title='Unsupervised anomaly detection in CS CSSR')
fig.add_trace(go.Scatter(x=iso_anomaly_df["index"].to_list(), y=iso_anomaly_df["CS CSSR"].to_list(),
                            mode='markers', name='anomalies'))
fig.update_xaxes(rangeslider_visible=True)
fig

In [49]:
from sklearn.neighbors import LocalOutlierFactor
lof = LocalOutlierFactor(n_neighbors=15, contamination=0.025)
y_pred = lof.fit_predict(x)
y_pred = [1 if x == -1 else 0 for x in y_pred]
y_pred[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [50]:
df["anomaly"] = y_pred
df.head()

,index,Date,RBS Id,TRAFFIC PS(DL+UL)(Gbits),HSDPA Data Volume,HSUPA Data Volume,Cell Traffic Volume DL Speech (Erlang),CS CSSR,CS DROP Rate,PS Call Setup SR,PS DROP Rate 2,Availability,Soft Handover,Irat CS Handover SR,IRAT PS 2,throughput,anomaly
0,0,16-12-21,0,74.39,66.74,7.65,156.83,99.69,0.77,95.94,1.72,100.0,99.86,93.94,100.00,755.79,0
1,1,17-12-21,0,65.41,58.40,7.01,107.82,99.26,0.97,95.40,2.37,100.0,99.67,97.98,100.00,755.79,0
2,2,18-12-21,0,65.67,58.93,6.74,94.38,99.26,1.09,94.97,3.03,100.0,99.57,96.43,0.00,755.79,0
3,3,19-12-21,0,57.79,50.91,6.87,101.48,99.08,1.23,95.43,2.60,100.0,99.72,93.14,100.00,755.79,0
4,4,20-12-21,0,79.74,70.04,9.69,163.22,99.65,0.76,95.16,1.61,100.0,99.86,94.23,66.67,755.79,0


In [51]:
lof_anomaly_df = pd.DataFrame(df)
lof_anomaly_df = lof_anomaly_df.loc[lof_anomaly_df['anomaly'] == 1]
lof_anomaly_df.head()

,index,Date,RBS Id,TRAFFIC PS(DL+UL)(Gbits),HSDPA Data Volume,HSUPA Data Volume,Cell Traffic Volume DL Speech (Erlang),CS CSSR,CS DROP Rate,PS Call Setup SR,PS DROP Rate 2,Availability,Soft Handover,Irat CS Handover SR,IRAT PS 2,throughput,anomaly
24,24,01-09-22,0,72.46,61.93,10.53,89.79,98.41,3.48,93.11,4.73,96.95,98.69,92.60,100.00,490.41,1
25,25,01-10-22,0,64.63,57.21,7.42,110.09,98.29,3.87,89.49,3.79,97.88,99.60,95.49,100.00,670.35,1
56,56,02-10-22,0,79.22,69.52,9.71,170.49,99.77,0.83,96.12,1.22,100.00,99.77,95.83,50.00,817.07,1
106,106,16-01-22,1,88.22,78.74,9.47,378.45,99.83,0.18,97.81,0.70,100.00,100.06,94.85,66.67,804.15,1
164,164,30-12-21,2,61.17,51.81,9.36,730.76,99.80,0.11,97.34,0.75,100.00,100.14,97.29,66.67,681.13,1


In [52]:
fig = px.line(df, x='index', y='CS CSSR', title='Unsupervised anomaly detection in CS CSSR')
fig.add_trace(go.Scatter(x=lof_anomaly_df["index"].to_list(), y=lof_anomaly_df["CS CSSR"].to_list(),
                        mode='markers', name='anomalies'))
fig.update_xaxes(rangeslider_visible=True)
fig

In [59]:
from sklearn.svm import OneClassSVM
ocsvm = OneClassSVM(kernel='rbf',
                     gamma="auto",
                     nu=0.1)
y_pred = ocsvm.fit_predict(x)
y_pred = [1 if x == -1 else 0 for x in y_pred]
y_pred[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [60]:
df["anomaly"] = y_pred
df.head()

,index,Date,RBS Id,TRAFFIC PS(DL+UL)(Gbits),HSDPA Data Volume,HSUPA Data Volume,Cell Traffic Volume DL Speech (Erlang),CS CSSR,CS DROP Rate,PS Call Setup SR,PS DROP Rate 2,Availability,Soft Handover,Irat CS Handover SR,IRAT PS 2,throughput,anomaly
0,0,16-12-21,0,74.39,66.74,7.65,156.83,99.69,0.77,95.94,1.72,100.0,99.86,93.94,100.00,755.79,0
1,1,17-12-21,0,65.41,58.40,7.01,107.82,99.26,0.97,95.40,2.37,100.0,99.67,97.98,100.00,755.79,0
2,2,18-12-21,0,65.67,58.93,6.74,94.38,99.26,1.09,94.97,3.03,100.0,99.57,96.43,0.00,755.79,0
3,3,19-12-21,0,57.79,50.91,6.87,101.48,99.08,1.23,95.43,2.60,100.0,99.72,93.14,100.00,755.79,0
4,4,20-12-21,0,79.74,70.04,9.69,163.22,99.65,0.76,95.16,1.61,100.0,99.86,94.23,66.67,755.79,0


In [61]:
ocsvm_anomaly_df = pd.DataFrame(df)
ocsvm_anomaly_df = ocsvm_anomaly_df.loc[ocsvm_anomaly_df['anomaly'] == 1]
ocsvm_anomaly_df.head()

,index,Date,RBS Id,TRAFFIC PS(DL+UL)(Gbits),HSDPA Data Volume,HSUPA Data Volume,Cell Traffic Volume DL Speech (Erlang),CS CSSR,CS DROP Rate,PS Call Setup SR,PS DROP Rate 2,Availability,Soft Handover,Irat CS Handover SR,IRAT PS 2,throughput,anomaly
9,9,25-12-21,0,60.16,52.90,7.26,118.15,98.97,2.93,95.07,2.83,98.01,99.52,95.97,66.67,645.25,1
16,16,01-01-22,0,68.80,59.75,9.05,98.97,99.00,3.71,95.40,3.64,96.73,99.51,94.93,100.00,634.37,1
23,23,01-08-22,0,76.68,67.55,9.14,123.39,98.87,3.00,94.32,3.23,97.75,99.42,96.14,40.00,628.69,1
24,24,01-09-22,0,72.46,61.93,10.53,89.79,98.41,3.48,93.11,4.73,96.95,98.69,92.60,100.00,490.41,1
25,25,01-10-22,0,64.63,57.21,7.42,110.09,98.29,3.87,89.49,3.79,97.88,99.60,95.49,100.00,670.35,1


In [62]:
fig = px.line(df, x='index', y='CS CSSR', title='Unsupervise anomaly detection in CS CSSR')
fig.add_trace(go.Scatter(x=ocsvm_anomaly_df["index"].to_list(),
                        y=ocsvm_anomaly_df["CS CSSR"].to_list(),
                        mode='markers',
                        name='anomaly'))
fig.update_xaxes(rangeslider_visible=True)
fig